In [1]:
import ribopy
from ribopy import Ribo
from functions import find_sequence, get_cds_range_lookup
import matplotlib.pyplot as plt
import numpy as np

In [2]:
ribo_path = f'/home/reiko/ribopy_analysis/mouse/all.ribo'
reference_path = '/home/reiko/ribopy_analysis/mouse/appris_mouse_v2_selected.fa.gz'
experiments = ['WT_control_A', 'WT_10min_A', 'WT_30min_A', 'WT_1hr_A']
min_len = 25
max_len = 31
alias = True

ribo_object = Ribo(ribo_path, alias = ribopy.api.alias.apris_human_alias)
sequence = find_sequence(ribo_object, reference_path)
cds_range = get_cds_range_lookup(ribo_object)

# Length distribution plots

In [3]:
length_dist = ribo_object.get_length_dist(region_name = "CDS")
selected_data = length_dist.loc[:, experiments]

# Plot the data
plt.figure(figsize=(10, 8))
for experiment in experiments:
    plt.plot(selected_data.index, selected_data[experiment], label=experiment)

plt.xlabel('Read Length', fontsize=18)
plt.ylabel('Frequency', fontsize=18)
plt.tick_params(axis='both', which='major', labelsize=16)  # Set the font size for tick labels
plt.legend(fontsize=16)
plt.savefig('/home/reiko/ribopy_analysis/mouse/output/length_dist.png')

# Metagene plots

In [4]:
metagene = ribo_object.get_metagene(site_type="start", range_lower = 25, range_upper = 31)
selected_col = [i for i in range(-50,51)]
metagene_selected = metagene[selected_col]

In [5]:
selected_rows = metagene_selected.loc[experiments]
selected_rows = selected_rows.T

# Plot the data
plt.figure(figsize=(10, 8))  # Adjust the figure size as needed
for col in selected_rows.columns:
    plt.plot(selected_rows.index, selected_rows[col], label=col)

plt.xlabel('Position', fontsize=18)
plt.ylabel('Frequency', fontsize=18)
plt.tick_params(axis='both', which='major', labelsize=16)
plt.legend(fontsize=16)
plt.savefig('/home/reiko/ribopy_analysis/mouse/output/metagene.png')

# Reads per nucleotide plots

In [6]:
ribo_object.get_metagene(site_type="start")

,-50,-49,-48,-47,-46,-45,-44,-43,-42,-41,...,41,42,43,44,45,46,47,48,49,50
experiment,,,,,,,,,,,,,,,,,,,,,
WT_10min_A,970,867,790,870,875,672,802,853,946,948,...,960,6146,1613,2016,6115,1873,1770,6381,1602,1377
WT_10min_B,338,310,398,424,341,251,419,334,355,354,...,657,3282,912,629,701,419,576,961,513,639
WT_10min_C,253,230,254,213,196,187,213,252,198,204,...,422,896,344,421,552,289,409,615,330,446
WT_1hr_A,575,603,552,603,585,477,581,613,620,670,...,729,3614,998,1871,3923,1089,1131,3445,883,882
WT_1hr_B,156,155,147,190,132,167,170,194,147,150,...,321,1473,358,336,546,235,279,587,266,420
WT_1hr_C,511,490,620,594,481,538,541,569,545,499,...,1226,1411,1172,1457,1999,964,1266,1361,878,1081
WT_30min_A,211,265,210,215,225,200,246,274,314,258,...,326,2198,487,354,1782,393,319,1985,491,296
WT_30min_B,8,11,13,13,5,6,5,8,13,12,...,15,17,15,9,12,13,17,12,5,11
WT_30min_C,415,365,416,480,357,394,426,406,358,362,...,769,1102,769,856,1145,634,758,976,657,679


In [7]:
region_counts = ribo_object.get_region_counts(experiments    = experiments,
                                              region_name    = "CDS",
                                              range_lower    = 25,
                                              range_upper    = 31,
                                              sum_lengths    = True,
                                              sum_references = False,
                                              alias          = False)

In [8]:
# Initialize a list to store the reads per nucleotide
reads_per_nucleotide_list = []

# Iterate over each transcript in the cds_range dictionary
for transcript, (start, end) in cds_range.items():
    # Get the length of the CDS region
    cds_length = end - start + 1

    # Get the counts for the CDS region for the current transcript
    cds_counts = region_counts.loc[transcript]
    
    # Calculate the total counts for the CDS region
    total_counts = cds_counts.sum()
    
    # Calculate the reads per nucleotide for the CDS region
    reads_per_nucleotide = total_counts / cds_length
    
    # Append the reads per nucleotide to the list
    reads_per_nucleotide_list.append(reads_per_nucleotide)

In [9]:
plt.figure(figsize=(10, 8))
plt.hist(reads_per_nucleotide_list, bins=20)
plt.yscale('log')
plt.xlabel('Reads per Nucleotide', fontsize=18)
plt.ylabel('Frequency (log scale)', fontsize=18)
plt.savefig('/home/reiko/ribopy_analysis/mouse/output/reads_per_nt.png')